<a href="https://colab.research.google.com/github/is0280fp/google_colab/blob/use_image_as_input/training_spectrogram.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive 
drive.mount('/content/drive')

Mounted at /content/drive


In [100]:
# ラベルの読み込み

with open("/content/drive/MyDrive/pickle/label_five_class.pickle", mode="rb") as f:
   label = pickle.load(f)
f.close()

In [101]:
# データセット内訳チェック
print(np.array(np.where(label == 0)).shape)
print(np.array(np.where(label == 1)).shape)
print(np.array(np.where(label == 2)).shape)
print(np.array(np.where(label == 3)).shape)
print(np.array(np.where(label == 4)).shape)

# NaNチェック
print(np.isnan(y).sum())

(1, 7755)
(1, 12748)
(1, 19873)
(1, 21386)
(1, 13970)
0


In [102]:
label.shape

(75732,)

In [ ]:
from sklearn.model_selection import train_test_split

# データ(ラベル)の分割
for i in range(10):
    train_label, test_label= train_test_split(label, train_size=9, test_size=1, random_state=42)
    print(i,"回目")
    print(train_label.shape)
    print(test_label.shape)

In [ ]:
import tensorflow as tf
from tensorflow.keras import optimizers, regularizers
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, LSTM, InputLayer, Dense, Dropout, Activation, Flatten, concatenate, Conv1D, MaxPooling1D, Input, Reshape, Bidirectional
from keras.models import Model
from tensorflow.keras.callbacks import TensorBoard, LearningRateScheduler
import time
import math
from keras.callbacks import ModelCheckpoint, EarlyStopping
from sklearn.metrics import confusion_matrix, classification_report, plot_confusion_matrix
import seaborn as sns
from sklearn.model_selection import train_test_split

# NAME = "HC-vs-C-LSTM_2class"
# NAME = "HC-vs-A-LSTM_2class"
# NAME = "HC-vs-K-LSTM_2class"
# NAME = "HC-vs-H-LSTM_2class"
# NAME = "HC-vs-GDs-LSTM_2class"
# NAME = "H-vs-K-vs-A-vs-C-LSTM_4class"
#NAME = "HC-vs-H-vs-K-vs-A-vs-C-LSTM_5class"
NAME = "H-vs-K-vs-A-vs-C-CNN_4class"
# NAME = "HC-vs-Patients-CNN_5class"
# NAME = "HC-vs-GDs-CNN_2class"
# NAME = "HC-vs-H-CNN_2class"
# NAME = "HC-vs-K-CNN_2class"
# NAME = "HC-vs-A-CNN_2class"
# NAME = "HC-vs-C-CNN_2class"

#-----------------------------------初期設定-------------------------------------------
# tensorboardのログ
tensorboard = TensorBoard(log_dir="logs/{}".format(NAME))

# 学習打ち切り目安
early_stopping = EarlyStopping(
                        monitor='val_loss',
                        min_delta=0.0,
                        patience=10,
                )

# checkpointの設定
checkpoint = ModelCheckpoint(
                    filepath="saved_model/{}".format(NAME),
                    monitor='val_loss',
                    save_best_only=True,
                    period=1,
                )

# weightの初期化
initializer = tf.initializers.he_normal()

#-----------------------------------学習モデル作成-------------------------------------------


#-----------------------------------Optimizer-------------------------------------------
# Optimizer: 多クラス分類+Adam
# model.compile(loss='sparse_categorical_crossentropy', 
#               optimizer= optimizers.Adam(lr=0.002, beta_1= 0.9, beta_2= 0.999),
#               metrics=['accuracy'],
#               )

# Optimizer: 多クラス分類+SGD
model.compile(loss='sparse_categorical_crossentropy',
              optimizer= optimizers.SGD(lr=5e-2, decay=1e-2),
              metrics=['accuracy'],
              )

# Optimizer: 2クラス分類
# model.compile(loss='binary_crossentropy',
#               optimizer= optimizers.SGD(lr=5e-3),
#               metrics=['accuracy'],
#               )

#-----------------------------------summer出力-------------------------------------------
model.summary()

#-----------------------------------学習-------------------------------------------
model.fit(X, train_label,
          batch_size=5,
          epochs=1000,
          validation_split=0.2,
          callbacks=[tensorboard])

# training parameters = kernel_size * (1Dだったらここは1, 2Dだったらここはkernel_size) * output_channels + num_bias
# 216 = 8 * 1 * 24 + 24

In [ ]:
# データの読み込み 
# yesを選ぶように!
!unzip /content/drive/MyDrive/zip_spectrogram_data/AP_left.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/AP_right.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/AP_right_69826.zip

In [ ]:
# データの読み込み
# yesを選ぶように!
!unzip /content/drive/MyDrive/zip_spectrogram_data/ML_left.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/ML_left_58106.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/ML_right.zip

In [ ]:
# データの読み込み
# yesを選ぶように!
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_left.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_left_48327.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_right.zip
!unzip /content/drive/MyDrive/zip_spectrogram_data/V_right_34533.zip

In [96]:
metadata_for_GRF.shape

(75732, 6)